In [1]:
# Run code on Anaconda Env.
# Construct data source by downloading 2018 data
import folktables
from folktables import ACSDataSource, ACSIncome
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)  # have to use 'download=True' if data not already avaiable locally

features, label, group = ACSIncome.df_to_numpy(acs_data)  # split data into corresponding features, labels, and group



/Users/timothychang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/timothychang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def mixRaceRates(data, target_bhn_ratio):
    # Input: complete training dataset as numpy array
    # Output: filtered dataset where ratio of bhn indiviudals satisfies the desired target (+/- 1%)
    # RAC1P column # for ASCIncome task -> 9

    # count occurances of BHN in the RAC1P column
    rac1p_values = data[:, 9]
    bhn_count = np.count_nonzero(rac1p_values == 2) + np.count_nonzero(rac1p_values == 3)
    
    # calc current ratio
    current_ratio = bhn_count / len(rac1p_values)

    # if the current ratio is within 5% of desired ratio, return the dataset
    if abs(current_ratio - target_bhn_ratio) < 0.01:
        print("here")
        return data

    # if current ratio > target ratio: remove bhn rows
    if current_ratio > target_bhn_ratio:
        # find indicies of bhn
        target_indicies = np.where((rac1p_values == 2) | (rac1p_values == 3))[0]
        np.random.shuffle(target_indicies)
        # calculate num of bhn rows to remove, then remove specified # of bhn rows
        remove_count = bhn_count - int(target_bhn_ratio * len(rac1p_values))
        data = np.delete(data, target_indicies[:remove_count], axis=0)
    else: 
    # current ratio < target ratio: remove non-bhn rows
        # find indicies of non-bhn
        target_indicies = np.where((rac1p_values != 2) & (rac1p_values != 3))[0]
        np.random.shuffle(target_indicies)
        # calculate num of non-bhn rows to remove, then remove specified # of non-bhn rows
        non_bhn_count = len(rac1p_values) - (np.count_nonzero(rac1p_values == 2) + np.count_nonzero(rac1p_values == 3))
        remove_count = non_bhn_count - int((1-target_bhn_ratio) * len(rac1p_values))
        data = np.delete(data, target_indicies[:remove_count], axis=0)
        
    # recalc current ratio
    '''
    rac1p_values = data[:, 9]
    bhn_count = np.count_nonzero(rac1p_values == 2) + np.count_nonzero(rac1p_values == 3)
    current_ratio = bhn_count / len(rac1p_values)
    print("updated ratio: ", current_ratio)
    '''

    return data


In [3]:
# randomly split the data into training and testing
# train-test split: 80/20
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features, label, group, test_size=0.2)

# reshape y_train into column vector, then concatenate with X_train to reformat training data
train_data = np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1)

# filter training data to satisfy desired BHN ratio
modified_train_data = mixRaceRates(train_data, 0.03)
new_X_train = modified_train_data[:, :-1] # get all cols except for last one
new_y_train = modified_train_data[:, -1]  # get only the last col

# create the pipeline: normalize data, then use logistic regression as classifier
model = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=16, min_samples_leaf=3))
# train the model with the training data
model.fit(new_X_train, new_y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=16, min_samples_leaf=3))])

In [4]:
# make predictions on test data set
yhat = model.predict(X_test)

yhat_wa = yhat[(group_test == 1) | (group_test == 6)]  # all rows from prediction where group=1 (white) or group=6 (asian)
yhat_bhn = yhat[(group_test == 2) | (group_test == 3)]  # all rows from prediction where group=2 (black) or group=3 (american indian)

y_test_wa = y_test[(group_test == 1) | (group_test == 6)]  # all rows from test set where group=1 (white) or group=6 (asian)
y_test_bhn = y_test[(group_test == 2) | (group_test == 3)]  # all rows from test set where group=2 (black) or group=3 (american indian)


In [5]:

# Find tpr, fpr, tnr, fnr with buit-in confusion matrix for binary labels
def confusion(pred, label):
    tn, fp, fn, tp = confusion_matrix(label, pred).ravel()
    print("Total Accuracy: ", (tp + tn)/ (tp + tn + fp + fn))
    print("Total True Positive Rate: ", tp / (tp + fn))
    print("Total False Negative Rate: ", fn / (tp + fn))

print("Metrics for WA population:")
confusion(yhat_wa, y_test_wa)
print("\nMetrics for BHN population:")
confusion(yhat_bhn, y_test_bhn)

#print("\n% dataset WA: ", (np.count_nonzero(group==1) + np.count_nonzero(group==6)) / group.shape[0] * 100)
#print("% dataset BHN: ", (np.count_nonzero(group==2) + np.count_nonzero(group==3))/ group.shape[0] * 100)

Metrics for WA population:
Total Accuracy:  0.8182171169713639
Total True Positive Rate:  0.8054704751050644
Total False Negative Rate:  0.19452952489493552

Metrics for BHN population:
Total Accuracy:  0.794102431453699
Total True Positive Rate:  0.6793893129770993
Total False Negative Rate:  0.32061068702290074
